In [1]:
import sys
import os

os.chdir('../')
sys.path.append('../')

In [2]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from tutorial_dataset import MyDataset
from cldm.logger import ImageLogger
from cldm.model import create_model, load_state_dict

2023-04-18 10:37:19.611613: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-18 10:37:19.662243: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-18 10:37:20.877661: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
resume_path = '/home/aiteam/tykim/generative_model/framework/diffusion/ControlNet/test.ckpt'
batch_size = 4
logger_freq = 300
learning_rate = 1e-5
sd_locked = True
only_mid_control = False


In [4]:
model = create_model('./models/cldm_v15.yaml').cpu()
model.load_state_dict(load_state_dict(resume_path, location='cpu'))
model.learning_rate = learning_rate
model.sd_locked = sd_locked
model.only_mid_control = only_mid_control

No module 'xformers'. Proceeding without it.
ControlLDM: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.22.mlp.fc1.bias', 'vision_model.encoder.layers.10.mlp.fc1.weight', 'vision_model.encoder.layers.19.layer_norm1.weight', 'vision_model.encoder.layers.9.mlp.fc1.bias', 'vision_model.encoder.layers.6.layer_norm2.bias', 'vision_model.encoder.layers.1.self_attn.v_proj.weight', 'vision_model.encoder.layers.18.self_attn.q_proj.weight', 'vision_model.encoder.layers.18.self_attn.v_proj.weight', 'vision_model.encoder.layers.10.layer_norm2.bias', 'vision_model.post_layernorm.bias', 'vision_model.encoder.layers.7.self_attn.q_proj.weight', 'vision_model.encoder.layers.10.self_attn.k_proj.weight', 'vision_model.encoder.layers.0.self_attn.k_proj.bias', 'vision_model.encoder.layers.8.layer_norm2.bias', 'vision_model.encoder.layers.0.mlp.fc1.weight', 'vision_model.encoder.layers.5.mlp.fc2.bias', 'vision_model.encoder.layers.0.layer_norm1.weight', 'vision_mod

Loaded model config from [./models/cldm_v15.yaml]
Loaded state_dict from [/home/aiteam/tykim/generative_model/framework/diffusion/ControlNet/test.ckpt]


In [5]:
dataset = MyDataset()

In [6]:
dataset[0]

{'jpg': array([[[0.9843137 , 0.92156863, 0.8039216 ],
         [0.9843137 , 0.92156863, 0.8039216 ],
         [0.9843137 , 0.92156863, 0.8039216 ],
         ...,
         [0.9843137 , 0.92156863, 0.8039216 ],
         [0.9843137 , 0.92156863, 0.8039216 ],
         [0.9843137 , 0.92156863, 0.8039216 ]],
 
        [[0.9843137 , 0.92156863, 0.8039216 ],
         [0.9843137 , 0.92156863, 0.8039216 ],
         [0.9843137 , 0.92156863, 0.8039216 ],
         ...,
         [0.9843137 , 0.92156863, 0.8039216 ],
         [0.9843137 , 0.92156863, 0.8039216 ],
         [0.9843137 , 0.92156863, 0.8039216 ]],
 
        [[0.9843137 , 0.92156863, 0.8039216 ],
         [0.9843137 , 0.92156863, 0.8039216 ],
         [0.9843137 , 0.92156863, 0.8039216 ],
         ...,
         [0.9843137 , 0.92156863, 0.8039216 ],
         [0.9843137 , 0.92156863, 0.8039216 ],
         [0.9843137 , 0.92156863, 0.8039216 ]],
 
        ...,
 
        [[0.9843137 , 0.92156863, 0.8039216 ],
         [0.9843137 , 0.92156863, 

In [7]:
dataloader = DataLoader(dataset, num_workers=2, batch_size=2, shuffle=False)

In [10]:
batch = next(iter(dataloader))

In [13]:
batch['jpg'].shape

torch.Size([2, 512, 512, 3])

In [16]:
batch['jpg'].device

device(type='cpu')

In [18]:
model.device

device(type='cpu')

In [29]:
batch['jpg'] = batch['jpg'].to('cuda')
batch['hint'] = batch['hint'].to('cuda')

In [4]:
import open_clip
import torch
from PIL import Image

In [5]:
arch="ViT-H-14"
version="laion2b_s32b_b79k"

model, _, preprocess = open_clip.create_model_and_transforms(arch, device=torch.device('cpu'), pretrained=version)

In [13]:
type(model)

open_clip.model.CLIP

In [6]:
image = preprocess(Image.open("/home/aiteam/tykim/generative_model/framework/diffusion/ControlNet/training/fill50k/target/2.png")).unsqueeze(0)

In [9]:
image.shape

torch.Size([1, 3, 224, 224])

In [10]:
image_features = model.encode_image(image)

In [11]:
image_features.shape

torch.Size([1, 1024])

In [12]:
type(model.visual)

open_clip.model.VisualTransformer

In [ ]:
with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

print("Label probs:", text_probs)  # prints: [[1., 0., 0.]]